In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install SimpleITK

     |████████████████████████████████| 42.5MB 97kB/s 


In [3]:
import SimpleITK as sitk
import numpy as np
import torch
from pathlib import Path
import matplotlib.pyplot as plt

In [4]:
import numpy as np
from scipy import ndimage

def zoom_data(data, voxel_size, is_label, factor=1.0, zoom_func=ndimage.zoom, **kwargs):
    '''
    :param 
        - data: volume or label
        - voxel_size: real distance of voxel - (depth, height, width)
        - factor: rate of size out with size in of volume - (a float number) 
        - order: type of spline interpolation
        - zoom_func: zoom function: defautl is scipy.ndimage.zoom()
        - karg: a dict of options to pass to zoom_func.
    '''
    if is_label:
        data[data>0] = 255

    # compute factor size
    zoom_factor = voxel_size[2] / factor , voxel_size[0] / factor, voxel_size[1] / factor

    # zoom data
    vol_out = zoom_func(data, zoom_factor, **kwargs)

    # Make width and height is a even number
    if vol_out.shape[1] % 2 == 1:
        vol_out = vol_out[:, 0:-1, :]
    if vol_out.shape[2] % 2 == 1:
        vol_out = vol_out[:, :, 0:-1]

    if is_label:
        vol_out[vol_out <= 100] = 0
        vol_out[vol_out > 100] = 1
        
    return vol_out

In [5]:
save_dir = 'PTH'
save_vol_dir = f'/content/drive/My Drive/Tumor_src/Datasets/SLIVER07/{save_dir}/VOLUME'
save_mask_dir = f'/content/drive/My Drive/Tumor_src/Datasets/SLIVER07/{save_dir}/LIVER'

save_vol_dir = Path(save_vol_dir)
save_mask_dir = Path(save_mask_dir)

save_vol_dir.mkdir(parents=True, exist_ok=True)
save_mask_dir.mkdir(parents=True, exist_ok=True)

save_vol_dir = str(save_vol_dir)
save_mask_dir = str(save_mask_dir)

In [6]:
zoom_spacing = 1.0

for idx in range(0, 20):
    vol_path = f'/content/drive/My Drive/Tumor_src/Datasets/SLIVER07/scan/liver-orig{idx+1:03d}.mhd'
    seg_path = f'/content/drive/My Drive/Tumor_src/Datasets/SLIVER07/label/liver-seg{idx+1:03d}.mhd'
    # Read the .nii image containing the volume with SimpleITK:
    volume = sitk.ReadImage(vol_path)
    seg = sitk.ReadImage(seg_path)
    spacing = seg.GetSpacing() # order h, w, d

    # and access the numpy array:
    vol_data = sitk.GetArrayFromImage(volume).astype(np.float32)
    seg_data = sitk.GetArrayFromImage(seg).astype(np.float32)
    
    # process
    seg_data[seg_data > 1] = 1 # merge tumor to liver

    # zoom factor
    # zoom_volume = zoom_data(vol_data, spacing, False, factor=zoom_spacing, zoom_func=ndimage.zoom, order=3)
    # zoom_mask = zoom_data(seg_data, spacing, True, factor=zoom_spacing, zoom_func=ndimage.zoom, order=1)
    zoom_volume = vol_data
    zoom_mask = seg_data

    # save as pth file
    vol_name = f'/volume_{idx}.pth'
    torch.save(torch.from_numpy(zoom_volume).to(torch.float32), save_vol_dir + vol_name)
    torch.save(torch.from_numpy(zoom_mask).to(torch.float32), save_mask_dir + vol_name)
    print(f'Save vol_{idx}')

print('Done!')

Save vol_0
Save vol_1
Save vol_2
Save vol_3
Save vol_4
Save vol_5
Save vol_6
Save vol_7
Save vol_8
Save vol_9
Save vol_10
Save vol_11
Save vol_12
Save vol_13
Save vol_14
Save vol_15
Save vol_16
Save vol_17
Save vol_18
Save vol_19
Done!
